EL REPO ES ENCONTRADO ACA: https://github.com/DiegoLinares11/Lab8-Databricks 


DIEGO LINARES


JOSE PRINCE

1. Importar librerías y definir funciones base

In [0]:
%pip install openpyxl
%pip install pyreadstat

In [0]:
%restart_python

## Carga desde archivo y construcción dicts

In [0]:
import os, io
import pandas as pd
import pyreadstat

# Importante: pandas SI puede leer rutas locales si usas el prefijo /dbfs/...
DIRS = [
    "/Volumes/workspace/default/fallecidos_lesionados/",
    "/Volumes/workspace/default/hechos_transito/",
    "/Volumes/workspace/default/vehiculos_involucrados/",
]

for d in DIRS:
    for fi in dbutils.fs.ls(d):
        file_path = fi.path                      # ej: /Volumes/workspace/.../archivo.xlsx
        file_path_dbfs = "/dbfs" + file_path     # ruta local (driver)
        lower = fi.name.lower()
        if lower.endswith(".xlsx"):
            out_csv = file_path.replace(".xlsx", ".csv")
            try:
                df = pd.read_excel(file_path_dbfs)   # lee Excel
                df.to_csv("/dbfs" + out_csv, index=False)
                print(f"XLSX→CSV: {fi.name}")
                dbutils.fs.rm(file_path)  # opcional: borra el Excel
            except Exception as e:
                print("Error XLSX", fi.name, e)
        elif lower.endswith(".sav"):
            out_csv = file_path.replace(".sav", ".csv")
            try:
                df, meta = pyreadstat.read_sav(file_path_dbfs, apply_value_formats=False)
                df.to_csv("/dbfs" + out_csv, index=False)
                print(f"SAV→CSV: {fi.name}")
                dbutils.fs.rm(file_path)  # opcional
            except Exception as e:
                print("Error SAV", fi.name, e)


Unir TODOS los CSV de una carpeta en un solo DataFrame (robusto a columnas distintas)

In [0]:
from pyspark.sql import functions as F

DIR_HECHOS     = "/Volumes/workspace/default/hechos_transito"
DIR_VEHICULOS  = "/Volumes/workspace/default/vehiculos_involucrados"
DIR_FALLECIDOS = "/Volumes/workspace/default/fallecidos_lesionados"

def read_folder_csv_uc(path: str):
    df = (spark.read
          .option("header", True)
          .option("inferSchema", True)
          .csv(f"{path}/*.csv"))
    
    # UC: usa _metadata.file_path en lugar de input_file_name()
    if "_metadata" in df.columns:
        df = df.withColumn("source_file", F.col("_metadata.file_path"))
    else:
        # Fallback UC (si por algún motivo no aparece _metadata)
        df = df.withColumn("source_file", F.lit(None))
    return df

DIR_HECHOS     = "/Volumes/workspace/default/hechos_transito"
DIR_VEHICULOS  = "/Volumes/workspace/default/vehiculos_involucrados"
DIR_FALLECIDOS = "/Volumes/workspace/default/fallecidos_lesionados"

hechos_raw     = read_folder_csv_uc(DIR_HECHOS)
vehiculos_raw  = read_folder_csv_uc(DIR_VEHICULOS)
fallecidos_raw = read_folder_csv_uc(DIR_FALLECIDOS)

In [0]:
# Canoniza nombres (alias → nombre estándar)
ALIASES = {
    "num_hecho":      ["num_hecho","num","num_correl","núm_corre","num_corre","num_correlativo"],
    "anio":           ["anio","año","ano","anio_ocu","año_ocu","ano_ocu"],
    "mes":            ["mes","mes_ocu"],
    "dia":            ["dia","día","dia_ocu","día_ocu"],
    "dia_sem":        ["dia_sem","día_sem","dia_sem_ocu","día_sem_ocu"],
    "hora":           ["hora","hora_ocu"],
    "g_hora":         ["g_hora","g_hora_5"],
    "depto":          ["depto","depto_ocu"],
    "mupio":          ["mupio","mupio_ocu","muni_ocu","municipio"],
    "zona":           ["zona","zona_ocu"],
    "area":           ["area","areag_ocu","area_ocu"],
    "tipo_accidente": ["tipo_accidente","tipo_eve","tipo_evento","tipo_acc","tipo"],
    "causa_acc":      ["causa_acc","causa"],
    "sexo_pil":       ["sexo_pil","sexo_piloto","sexo","sexo_per"],
    "edad_pil":       ["edad_pil","edad","edad_piloto","edad_per"],
    "g_edad":         ["g_edad","g_edad_2","g_edad_80ymás","g_edad_60ymás","edad_quinquenales"],
    "mayor_menor":    ["mayor_menor"],
    "tipo_veh":       ["tipo_veh","tipo_vehiculo"],
    "marca_veh":      ["marca_veh","marca"],
    "color_veh":      ["color_veh","color"],
    "modelo_veh":     ["modelo_veh","modelo"],
    "g_modelo_veh":   ["g_modelo_veh"],
    "estado_pil":     ["estado_pil","estado_piloto","estado","estado_con","fall_les"],
    "intencionalidad":["int_o_noint","intencionalidad"],
    "source_file":    ["source_file","_metadata.file_path","_source_file"]
}

def _first_present(cols, cands):
    lc = {c.lower(): c for c in cols}
    for x in cands:
        if x.lower() in lc: return lc[x.lower()]
    return None

def canonize(df):
    mapping = {}
    for canon, cands in ALIASES.items():
        src = _first_present(df.columns, cands)
        if src and src != canon:
            mapping[src] = canon
    for src, dst in mapping.items():
        df = df.withColumnRenamed(src, dst)
    return df

hechos_std     = canonize(hechos_raw)
vehiculos_std  = canonize(vehiculos_raw)
fallecidos_std = canonize(fallecidos_raw)

# Si falta 'anio' en alguna hoja, extraerlo del nombre de archivo
for name in ["hechos_std","vehiculos_std","fallecidos_std"]:
    df = locals()[name]
    if "anio" not in df.columns:
        df = df.withColumn("anio", F.regexp_extract("source_file", r"(20\d{2})", 1).cast("int"))
        locals()[name] = df


### 2) Decodificación con diccionario de datos (genérico)

In [0]:
from pyspark.sql import types as T

DICC_DIR = "/Volumes/workspace/default/diccionario" 

def load_map(colname: str):
    path = f"{DICC_DIR}/{colname}.csv"
    try:
        dim = (spark.read.option("header", True).csv(path)
               .select(F.col("codigo").cast("string").alias("k"), F.col("etiqueta").alias("v")))
        # create_map exige lista [k1,v1,k2,v2,...]
        pairs = [F.lit(x) for r in dim.collect() for x in (r["k"], r["v"])]
        return F.create_map(pairs) if pairs else None
    except Exception:
        return None

# Fallback de departamentos (por si no cargas diccionario aún)
CODIGOS_DEPARTAMENTOS = {
  "1":"Guatemala","2":"El Progreso","3":"Sacatepéquez","4":"Chimaltenango","5":"Escuintla",
  "6":"Santa Rosa","7":"Sololá","8":"Totonicapán","9":"Quetzaltenango","10":"Suchitepéquez",
  "11":"Retalhuleu","12":"San Marcos","13":"Huehuetenango","14":"Quiché","15":"Baja Verapaz",
  "16":"Alta Verapaz","17":"Petén","18":"Izabal","19":"Zacapa","20":"Chiquimula",
  "21":"Jalapa","22":"Jutiapa","99":"Desconocido"
}
depto_map_fallback = F.create_map([F.lit(x) for kv in CODIGOS_DEPARTAMENTOS.items() for x in kv])

def decode_column(df, colname):
    # intenta usar diccionario; si no existe y la columna es 'depto', usa fallback
    m = load_map(colname)
    if m is None and colname == "depto":
        m = depto_map_fallback
    if m is None or colname not in df.columns:
        return df
    # normaliza a string y aplica mapeo
    return (df
      .withColumn(colname, 
          m.getItem(
              F.when(F.col(colname).isNull(), F.lit(None))
               .otherwise(F.regexp_replace(F.col(colname).cast("string"), r"\.0$", "")) # '1.0'->'1'
          )
      ))

# Aplica decodificación a las columnas relevantes
DECODE_COLS = ["depto","dia_sem","g_hora","tipo_accidente","estado_pil","intencionalidad",
               "tipo_veh","marca_veh","color_veh","g_edad","g_modelo_veh"]
for name in ["hechos_std","vehiculos_std","fallecidos_std"]:
    df = locals()[name]
    for c in DECODE_COLS:
        df = decode_column(df, c)
    locals()[name] = df


Tipado robusto (sin romper por rangos ‘2010-2019’)

In [0]:
from pyspark.sql import functions as F, types as T
from pyspark.sql.types import NullType
INT_LIKE = [
    "anio","mes","dia","hora","zona","mupio","area"
    # ojo: "modelo_veh" puede ser año numérico; castear pero limpiando '9999'
]

def to_int_safe(colname: str):
    # 1) a string y limpieza básica
    s = F.col(colname).cast("string")
    s = F.regexp_replace(F.trim(s), r"\s+", "")   # quita espacios
    s = F.regexp_replace(s, r"\.0$", "")          # '1.0' -> '1'
    # 2) solo deja dígitos enteros (con signo opcional)
    s = F.when(s.rlike(r"^-?\d+$"), s).otherwise(None)
    # 3) convierte a int
    return s.cast(T.IntegerType())

def clean_numeric(df):
    for c in INT_LIKE:
        if c in df.columns:
            df = df.withColumn(c, to_int_safe(c))
    if "modelo_veh" in df.columns:
        df = df.withColumn("modelo_veh", to_int_safe("modelo_veh"))
        df = df.withColumn("modelo_veh",
                           F.when(F.col("modelo_veh") == 9999, None)
                            .otherwise(F.col("modelo_veh")))
    return df

hechos_std     = clean_numeric(hechos_std)
vehiculos_std  = clean_numeric(vehiculos_std)
fallecidos_std = clean_numeric(fallecidos_std)

def ensure_source_file_string(df):
    if "source_file" in df.columns:
        df = df.withColumn(
            "source_file",
            F.coalesce(F.col("source_file").cast(T.StringType()), F.lit(""))
        )
    return df

hechos_std     = ensure_source_file_string(hechos_std)
vehiculos_std  = ensure_source_file_string(vehiculos_std)
fallecidos_std = ensure_source_file_string(fallecidos_std)

def drop_nulltype_cols(df):
    null_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, NullType)]
    return df.drop(*null_cols) if null_cols else df

hechos_std     = drop_nulltype_cols(hechos_std)
vehiculos_std  = drop_nulltype_cols(vehiculos_std)
fallecidos_std = drop_nulltype_cols(fallecidos_std)


In [0]:
from pyspark.sql import functions as F, types as T

YEAR_MIN, YEAR_MAX = 2013, 2024
MES_MIN,  MES_MAX  = 1, 12
DIA_MIN,  DIA_MAX  = 1, 31
HORA_MIN, HORA_MAX = 0, 23
ZONA_MIN,  ZONA_MAX = 1, 99

def sanitize_calendar(df, try_fix_year_from_source=True):
    if "anio" in df.columns:
        # anio dentro de rango, si no → NULL
        df = df.withColumn(
            "anio",
            F.when((F.col("anio") >= YEAR_MIN) & (F.col("anio") <= YEAR_MAX), F.col("anio"))
             .otherwise(F.lit(None).cast("int"))
        )
        # recuperar desde source_file, evitando cast de "" → int
        if try_fix_year_from_source and "source_file" in df.columns:
            y_str = F.regexp_extract(F.col("source_file"), r"(20\d{2})", 1)
            y_int = F.when(F.length(y_str) > 0, y_str.cast("int")) \
                     .otherwise(F.lit(None).cast("int"))
            df = df.withColumn("anio", F.coalesce(F.col("anio"), y_int))

    if "mes" in df.columns:
        df = df.withColumn("mes",  F.when((F.col("mes")  >= MES_MIN)  & (F.col("mes")  <= MES_MAX),  F.col("mes")))
    if "dia" in df.columns:
        df = df.withColumn("dia",  F.when((F.col("dia")  >= DIA_MIN)  & (F.col("dia")  <= DIA_MAX),  F.col("dia")))
    if "hora" in df.columns:
        df = df.withColumn("hora", F.when((F.col("hora") >= HORA_MIN) & (F.col("hora") <= HORA_MAX), F.col("hora")))
    if "zona" in df.columns:
        df = df.withColumn("zona", F.when((F.col("zona") >= ZONA_MIN) & (F.col("zona") <= ZONA_MAX), F.col("zona")))

    if "modelo_veh" in df.columns:
        df = df.withColumn(
            "modelo_veh",
            F.when((F.col("modelo_veh") >= 1900) & (F.col("modelo_veh") <= 2030), F.col("modelo_veh"))
        )

    return df


In [0]:
OUT_SILVER_HECHOS     = "/Volumes/workspace/default/hechos_transito/silver"
OUT_SILVER_VEHICULOS  = "/Volumes/workspace/default/vehiculos_involucrados/silver"
OUT_SILVER_FALLECIDOS = "/Volumes/workspace/default/fallecidos_lesionados/silver"

for df, outp in [
    (hechos_std, OUT_SILVER_HECHOS),
    (vehiculos_std, OUT_SILVER_VEHICULOS),
    (fallecidos_std, OUT_SILVER_FALLECIDOS),
]:
    (df.coalesce(1)
       .write.mode("overwrite")
       .parquet(outp))
    print("Silver →", outp)


In [0]:
hechos_silver     = spark.read.parquet(OUT_SILVER_HECHOS)
vehiculos_silver  = spark.read.parquet(OUT_SILVER_VEHICULOS)
fallecidos_silver = spark.read.parquet(OUT_SILVER_FALLECIDOS)


In [0]:
# Aplica a tus SILVER cargados:
hechos_silver     = sanitize_calendar(hechos_silver)
vehiculos_silver  = sanitize_calendar(vehiculos_silver)
fallecidos_silver = sanitize_calendar(fallecidos_silver)

In [0]:
# Agrega esto después de cargar tus silver para entender tu data
def analizar_completitud(df, nombre):
    print(f"\n===== {nombre} =====")
    total = df.count()
    print(f"Total registros: {total}")
    
    cols_clave = ["anio", "mes", "dia", "hora", "depto", "tipo_accidente"]
    for col in cols_clave:
        if col in df.columns:
            no_null = df.filter(F.col(col).isNotNull()).count()
            pct = (no_null/total)*100
            print(f"{col:20s}: {no_null:6d} ({pct:5.1f}%)")
    
    # Combinaciones comunes
    print("\nCombinaciones completas:")
    print(f"  anio+mes+depto:      {df.filter(F.col('anio').isNotNull() & F.col('mes').isNotNull() & F.col('depto').isNotNull()).count()}")
    print(f"  anio+mes+dia+hora:   {df.filter(F.col('anio').isNotNull() & F.col('mes').isNotNull() & F.col('dia').isNotNull() & F.col('hora').isNotNull()).count()}")

analizar_completitud(hechos_silver, "HECHOS")
analizar_completitud(vehiculos_silver, "VEHICULOS")
analizar_completitud(fallecidos_silver, "FALLECIDOS")

In [0]:
hechos_fecha_completa = hechos_silver.filter(
    F.col("anio").isNotNull() & 
    F.col("mes").isNotNull() & 
    F.col("dia").isNotNull()
)

# Vista 2: Registros con hora (más detallados)
hechos_con_hora = hechos_silver.filter(
    F.col("hora").isNotNull()
)

# Vista 3: Resumen por año-mes-depto (para joins)
hechos_mensual = hechos_silver.filter(
    F.col("anio").isNotNull() & 
    F.col("mes").isNotNull() & 
    F.col("depto").isNotNull()
)

Helpers reutilizables

### Preguntas a responder
1. Contar registros por tabla (long)

## #1 – Conteos, .show(), describe y summary (por tabla)

In [0]:
from pyspark.sql.types import NumericType
from pyspark.sql import functions as F
def ae_conteos_describe_summary(nombre: str, sdfs: dict, n_show: int = 5):
    print(f"\n===== {nombre.upper()} =====")
    total_registros = 0
    for key, sdf in sdfs.items():
        c = sdf.count()
        total_registros += c
        print(f"{key:20s} -> {c:6d} registros")
    print(f"TOTAL {nombre}: {total_registros}")

    if sdfs:
        first_key = list(sdfs.keys())[0]
        print(f"\n--- Ejemplo .show() :: {first_key} ---")
        sdfs[first_key].show(n_show, truncate=False, vertical=True)

        num_cols = [f.name for f in sdfs[first_key].schema.fields if isinstance(f.dataType, NumericType)]
        if num_cols:
            print(f"\n--- describe(numéricas) :: {first_key} ---")
            sdfs[first_key].select(*num_cols).describe().show(truncate=False, vertical=True)

            print(f"\n--- summary(numéricas) :: {first_key} ---")
            sdfs[first_key].select(*num_cols).summary(
                "count","mean","stddev","min","25%","50%","75%","max"
            ).show(truncate=False, vertical=True)
        else:
            print(f"\n--- {first_key}: no hay columnas numéricas detectadas ---")

ae_conteos_describe_summary("hechos",     {"hechos": hechos_silver})
ae_conteos_describe_summary("vehiculos",  {"vehiculos": vehiculos_silver})
ae_conteos_describe_summary("fallecidos", {"fallecidos": fallecidos_silver})


## #2 – Años disponibles por tabla y validación

In [0]:
EXPECTED_YEARS = set(range(2013, 2024))  # 2013..2023

def report_years_col(df, titulo: str):
    print(f"\n===== Verificación de años: {titulo} =====")
    found = sorted([r[0] for r in df.select("anio").dropna().distinct().collect()])
    found_set = set(found)
    missing = sorted(EXPECTED_YEARS - found_set)
    outside = sorted(y for y in found if y not in EXPECTED_YEARS)
    print(f"Encontrados: {found if found else '—'} | "
          f"faltantes vs 2013–2023: {missing if missing else 'ninguno'} | "
          f"fuera de rango: {outside if outside else 'ninguno'}")
    return found_set

years_hechos     = report_years_col(hechos_silver, "hechos")
years_vehiculos  = report_years_col(vehiculos_silver, "vehiculos")
years_fallecidos = report_years_col(fallecidos_silver, "fallecidos")

print("\n¿Coinciden los conjuntos de años (intersección)?")
intersection_all = years_hechos & years_vehiculos & years_fallecidos
print("Intersección común:", sorted(intersection_all) if intersection_all else "—")


### #3 – Valores distintos de 'tipo de accidente'   (buscamos columnas candidatas por nombre aproximado)

In [0]:
# ### #3 – Valores distintos de 'tipo de accidente'
import unicodedata, re
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

def _strip_accents(s: str) -> str:
    s = unicodedata.normalize("NFD", s)
    return "".join(ch for ch in s if unicodedata.category(ch) != "Mn")

def _norm_text(s):
    if s is None: return None
    s = str(s).strip()
    s = _strip_accents(s).lower()
    s = re.sub(r"\s+", " ", s)
    return s
    
YEAR_RX = r"^(19\d{2}|20\d{2})(\.0)?$"   # 1984 o 1984.0
INT_FLOAT_RX = r"^(\d+)(?:\.0)?$"        # 1.0 -> 1

def clean_tipo_accidente(df, col="tipo_accidente"):
    if col not in df.columns:
        return df

    s = F.trim(F.col(col).cast("string"))
    s = F.regexp_replace(s, INT_FLOAT_RX, r"\1")
    s = F.when(s.rlike(YEAR_RX), F.lit(None)).otherwise(s)

    s = F.when(s == "99", F.lit("Ignorado")).otherwise(s)

    fixes = {
        "Colisi?n":"Colisión", "Embarranc?":"Embarrancó",
        "Encunet?":"Encunetó", "Ca?da":"Caída"
    }
    s = F.coalesce(*[F.when(s == bad, F.lit(good)) for bad, good in fixes.items()] + [s])

    return df.withColumn(col, s)

hechos_silver     = clean_tipo_accidente(hechos_silver, "tipo_accidente")
vehiculos_silver  = clean_tipo_accidente(vehiculos_silver, "tipo_accidente")
fallecidos_silver = clean_tipo_accidente(fallecidos_silver, "tipo_accidente")
@F.udf(returnType=StringType())
def tipo_canon(val):
    n = _norm_text(val)
    if not n: return None
    syn = {
            "colisiones":"colision", "colision multiple":"colision",
            "choques":"choque", "derrapes":"derrape",
            "vuelcos":"vuelco", "embarranco multiple":"embarranco",
            "encunetamiento":"encuneto", "encunetado":"encuneto",
            "caidas":"caida", "no especificado":"ignorado",
            "sin dato":"ignorado", "na":"ignorado", "n/a":"ignorado"
    }
    return syn.get(n, n)

def find_tipo_col(df):
    cands = [c for c in df.columns if re.sub(r"[\s\-\_]+","", c.lower()) \
            in ("tipodeaccidente","tipoaccidente","tipoacc","tipo","tipoeve","tipoevento")]
    if cands: return cands[0]
    for c in df.columns:
        lc = c.lower()
        if "tipo" in lc and ("accid" in lc or "eve" in lc or "evento" in lc):
            return c
    return None

def distinct_tipos(df, nombre):
    col = find_tipo_col(df)
    if not col:
        print(f"[{nombre}] No encontré columna de tipo de accidente"); 
        return None, None

    print(f"\n[{nombre}] Columna detectada: {col}")

    print(f"\n[{nombre}] Distinct RAW (muestra hasta 100):")
    df.select(col).distinct().orderBy(col).show(100, truncate=False)

    print(f"\n[{nombre}] Distinct NORMALIZADOS + conteo:")
    norm = (df
            .withColumn("tipo_norm", tipo_canon(F.col(col)))
            .groupBy("tipo_norm")
            .count()
            .orderBy(F.desc("count"), F.asc("tipo_norm")))
    norm.show(100, truncate=False)
    return col, norm

col_hechos,     acc_hechos_norm     = distinct_tipos(hechos_silver, "hechos")
col_vehiculos,  acc_vehiculos_norm  = distinct_tipos(vehiculos_silver, "vehiculos")
col_fallecidos, acc_fallecidos_norm = distinct_tipos(fallecidos_silver, "fallecidos")




### #4 – # de departamentos únicos por base(detecta columna 'departamento' aproximada)

In [0]:
# Hechos
deptos_hechos = hechos_silver.select("depto").filter(F.col("depto").isNotNull()).distinct()
count_h = deptos_hechos.count()
print(f"HECHOS: {count_h} departamentos únicos")
deptos_hechos.orderBy("depto").show(25, truncate=False)

# Vehículos
deptos_vehiculos = vehiculos_silver.select("depto").filter(F.col("depto").isNotNull()).distinct()
count_v = deptos_vehiculos.count()
print(f"\nVEHÍCULOS: {count_v} departamentos únicos")
deptos_vehiculos.orderBy("depto").show(25, truncate=False)

# Fallecidos
deptos_fallecidos = fallecidos_silver.select("depto").filter(F.col("depto").isNotNull()).distinct()
count_f = deptos_fallecidos.count()
print(f"\nFALLECIDOS: {count_f} departamentos únicos")
deptos_fallecidos.orderBy("depto").show(25, truncate=False)

# BONUS: Departamentos que aparecen en las 3 bases
deptos_comunes = (deptos_hechos
    .intersect(deptos_vehiculos)
    .intersect(deptos_fallecidos))
print(f"\nDepartamentos comunes a las 3 bases: {deptos_comunes.count()}")
deptos_comunes.orderBy("depto").show(25, truncate=False)

5. ¿Cuál es el total de accidentes por año y departamento?

In [0]:
df_p5 = (hechos_silver
    .filter(F.col("anio").isNotNull() & F.col("depto").isNotNull())
    .groupBy("anio", "depto")
    .count()
    .withColumnRenamed("count", "total_accidentes")  # ← Renombra después
    .orderBy("anio", "depto"))

display(df_p5)

Databricks visualization. Run in Databricks to view.

6. ¿Qué día de la semana registra más accidentes en 2023?

In [0]:
# Pregunta 6
df_p6 = (hechos_silver
    .filter((F.col("anio") == 2023) & F.col("dia_sem").isNotNull())
    .groupBy("dia_sem")
    .count()  # ← esto crea columna "count"
    .orderBy(F.desc("count")))

display(df_p6)

Databricks visualization. Run in Databricks to view.

7. Mostrar la distribución de accidentes por hora del día en el municipio de 
Guatemala. Graficar en un histograma.  

In [0]:
df_hist = (hechos_silver
  .filter(F.col("hora").isNotNull() & (F.col("depto")=="Guatemala") & (F.col("mupio")==1))
  .select(F.col("hora").cast("int")))

display(df_hist)  


Databricks visualization. Run in Databricks to view.